In [1]:
import pandas as pd
import os
import numpy as np
import datetime as dt
from scipy.interpolate import interp1d

path = os.getcwd()

/opt/homebrew/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [24]:
col2dtype = {"mjo20E": float, "mjo70E": float, "mjo80E": float, "mjo100E": float, "mjo120E": float,
             "mjo140E": float, "mjo160E": float, "mjo120W": float, "mjo40W": float, "mjo10W": float,
             "year": int, "month": int, "day": float, "ninoNINO1+2": float, "ninoANOM": float,
             "ninoNINO3": float, "ninoANOM.1": float, "ninoNINO4": float, "ninoANOM.2": float,
             "ninoNINO3.4": float, "ninoANOM.3": float, "oniTOTAL": float,
             "oniANOM": float, "pdo": float, "pna": float, "soi_anom": float, "soi_sd": float,
             "forecast_year": float, "volume": float, "mean_gws_inst": float, "mean_rtzsm_inst": float,
             "mean_sfsm_inst": float, "station": float, "PREC_DAILY": float, "WTEQ_DAILY": float, "TAVG_DAILY": float,
             "TMAX_DAILY": float, "TMIN_DAILY": float, "SNWD_DAILY": float}
data = pd.read_csv(os.path.join("..", "pipeline", "transformed_vars.csv"),
                   dtype=col2dtype)

In [25]:
site_id_str = 'site_id_'
site_id_cols = [col for col in data.columns if 'site_id' in col]
data['site_id'] = data[site_id_cols] \
    .idxmax(axis='columns') \
    .apply(lambda x: x[x.find(site_id_str) + len(site_id_str):])
data = data.drop(columns=site_id_cols)

In [35]:
# Create integer dates to work with
data.day = data.day.apply(lambda x: 15*(x == -1))

# Reformulate ONIXXX from one hot encoding to date
oni_cols = [col for col in data.columns if 'oniSEAS' in col]
month_conversion_dictionary = {
'oniSEAS_AMJ': 5,
'oniSEAS_ASO': 9,
'oniSEAS_DJF': 1,
'oniSEAS_FMA': 3,
'oniSEAS_JAS': 8,
'oniSEAS_JFM': 2,
'oniSEAS_JJA': 7,
'oniSEAS_MAM': 4,
'oniSEAS_MJJ': 6,
'oniSEAS_NDJ': 12,
'oniSEAS_OND': 11,
'oniSEAS_SON': 10
}
data['oni_month'] = data[oni_cols] \
    .idxmax(axis='columns') \
    .apply(lambda x: month_conversion_dictionary[x])

data.month[data.month == -1]=data.oni_month[data.month == -1]
print((data.month > 0).all())
print((data.date > 0).all())



/var/folders/nh/8k546jq14l5gnwkjgxvch8180000gn/T/ipykernel_3699/3238780978.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.month[data.month == -1]=data.oni_month[data.month == -1]


In [36]:
date_cols = ['year', 'month', 'day']
data["date"] = pd.to_datetime(data[date_cols].applymap(int))
data = data.sort_values('date')

/var/folders/nh/8k546jq14l5gnwkjgxvch8180000gn/T/ipykernel_3699/297992395.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data["date"] = pd.to_datetime(data[date_cols].applymap(int))


ValueError: cannot assemble the datetimes: time data "19780100" doesn't match format "%Y%m%d", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [33]:
not_having_volume_nor_year = data[5369:12092].volume.isna()
print(data[5369:12098].year)
no_vol_or_year_data = data[5369:12092][not_having_volume_nor_year]
#print(no_vol_data)
print(data.site_id.unique())
american_vols = data[data.site_id=="x_american_river_folsom_lake"]
ids = no_vol_or_year_data.site_id.unique()
years = no_vol_or_year_data.year.unique()
print(ids)
print(years)

5369     1982
5370     1982
5371     1982
5372     1982
5373     1982
         ... 
12093    2022
12094    2022
12095    2022
12096    2022
12097    2023
Name: year, Length: 6729, dtype: int64
['x_animas_r_at_durango' 'y_hungry_horse_reservoir_inflow'
 'y_snake_r_nr_heise' 'y_pueblo_reservoir_inflow'
 'y_sweetwater_r_nr_alcova' 'y_missouri_r_at_toston'
 'y_animas_r_at_durango' 'y_yampa_r_nr_maybell' 'y_libby_reservoir_inflow'
 'y_boise_r_nr_boise' 'y_green_r_bl_howard_a_hanson_dam'
 'y_taylor_park_reservoir_inflow' 'y_dillon_reservoir_inflow'
 'y_ruedi_reservoir_inflow' 'y_fontenelle_reservoir_inflow'
 'y_weber_r_nr_oakley' 'y_san_joaquin_river_millerton_reservoir'
 'y_merced_river_yosemite_at_pohono_bridge' 'y_american_river_folsom_lake'
 'y_colville_r_at_kettle_falls' 'y_stehekin_r_at_stehekin'
 'y_detroit_lake_inflow' 'y_virgin_r_at_virtin' 'y_skagit_ross_reservoir'
 'y_boysen_reservoir_inflow' 'y_pecos_r_nr_pecos'
 'y_owyhee_r_bl_owyhee_dam' 'x_hungry_horse_reservoir_inflow'
 'x_sn